In [1]:
import os
os.chdir('/home/nmduy/CA683/Credit Project')
#from fancyimpute import IterativeImputer as MICE
import pandas as pd
import numpy as np
import random

In [2]:
data = pd.read_csv('data/filtered_application_train.csv')
data = data.drop(data.columns[0], axis=1)

In [3]:
# convert to positive
data['DAYS_BIRTH'] = abs(data['DAYS_BIRTH'])

data['DAYS_EMPLOYED'] = abs(data['DAYS_EMPLOYED'])

data['DAYS_ID_PUBLISH'] = abs(data['DAYS_ID_PUBLISH'])

data['DAYS_LAST_PHONE_CHANGE'] = abs(data['DAYS_LAST_PHONE_CHANGE'])

data['DAYS_REGISTRATION'] = abs(data['DAYS_REGISTRATION'])

# Assign NaN for wrong number
data['DAYS_EMPLOYED_ANOM'] = data["DAYS_EMPLOYED"] == 365243
data['DAYS_EMPLOYED'].replace({365243: np.nan}, inplace = True)

In [4]:
##### CONTINUOUS vs CATEGORICAL #####
categorical_count = data.select_dtypes('object').apply(pd.Series.nunique, axis = 0)
categorical_name = list(categorical_count.keys())
continuous_name = [x for x in list(data.columns) if x not in categorical_name]

In [5]:
continuous_name

['SK_ID_CURR',
 'TARGET',
 'NAME_CONTRACT_TYPE',
 'CODE_GENDER',
 'FLAG_OWN_CAR',
 'FLAG_OWN_REALTY',
 'CNT_CHILDREN',
 'AMT_INCOME_TOTAL',
 'AMT_CREDIT',
 'AMT_ANNUITY',
 'AMT_GOODS_PRICE',
 'REGION_POPULATION_RELATIVE',
 'DAYS_BIRTH',
 'DAYS_EMPLOYED',
 'DAYS_REGISTRATION',
 'DAYS_ID_PUBLISH',
 'OWN_CAR_AGE',
 'FLAG_MOBIL',
 'FLAG_EMP_PHONE',
 'FLAG_WORK_PHONE',
 'FLAG_CONT_MOBILE',
 'FLAG_PHONE',
 'FLAG_EMAIL',
 'CNT_FAM_MEMBERS',
 'REGION_RATING_CLIENT',
 'REGION_RATING_CLIENT_W_CITY',
 'HOUR_APPR_PROCESS_START',
 'REG_REGION_NOT_LIVE_REGION',
 'REG_REGION_NOT_WORK_REGION',
 'LIVE_REGION_NOT_WORK_REGION',
 'REG_CITY_NOT_LIVE_CITY',
 'REG_CITY_NOT_WORK_CITY',
 'LIVE_CITY_NOT_WORK_CITY',
 'EXT_SOURCE_2',
 'EXT_SOURCE_3',
 'OBS_30_CNT_SOCIAL_CIRCLE',
 'DEF_30_CNT_SOCIAL_CIRCLE',
 'OBS_60_CNT_SOCIAL_CIRCLE',
 'DEF_60_CNT_SOCIAL_CIRCLE',
 'DAYS_LAST_PHONE_CHANGE',
 'FLAG_DOCUMENT_2',
 'FLAG_DOCUMENT_3',
 'FLAG_DOCUMENT_4',
 'FLAG_DOCUMENT_5',
 'FLAG_DOCUMENT_6',
 'FLAG_DOCUMENT_7',
 'FL

In [6]:
categorical_name

['NAME_TYPE_SUITE',
 'NAME_INCOME_TYPE',
 'NAME_EDUCATION_TYPE',
 'NAME_FAMILY_STATUS',
 'NAME_HOUSING_TYPE',
 'OCCUPATION_TYPE',
 'WEEKDAY_APPR_PROCESS_START',
 'ORGANIZATION_TYPE']

In [7]:
##### ENCODE #####
encoded_train = pd.get_dummies(data)
encoded_train.to_csv('encoded_train.csv', index=False)

In [ ]:
##### REMOVE NAN #####
encoded_train_dropna = encoded_train.dropna(axis=0, how='any')
print(f"Before drop: {len(encoded_train)} rows\nAfter drop: {len(encoded_train_dropna)} rows")

encoded_train_dropna.to_csv('encoded_train_dropna.csv', index=False)

In [23]:
##### REMOVE FLAG FEATURE #####
Flag_ft_names = [x for x in list(data.columns) if 'FLAG' in x]
data_rm_flag = data.drop(Flag_ft_names, axis=1)
data_rm_flag.shape

(307511, 47)

In [24]:
data_rm_flag.to_csv('filtered_app_train_remove_flag.csv', index=False)

In [25]:
##### CONTINUOUS vs CATEGORICAL #####
categorical_count = data_rm_flag.select_dtypes('object').apply(pd.Series.nunique, axis = 0)
categorical_name = list(categorical_count.keys())
continuous_name = [x for x in list(data_rm_flag.columns) if x not in categorical_name]
print(categorical_name)
print(continuous_name)

['NAME_TYPE_SUITE', 'NAME_INCOME_TYPE', 'NAME_EDUCATION_TYPE', 'NAME_FAMILY_STATUS', 'NAME_HOUSING_TYPE', 'OCCUPATION_TYPE', 'WEEKDAY_APPR_PROCESS_START', 'ORGANIZATION_TYPE']
['SK_ID_CURR', 'TARGET', 'NAME_CONTRACT_TYPE', 'CODE_GENDER', 'CNT_CHILDREN', 'AMT_INCOME_TOTAL', 'AMT_CREDIT', 'AMT_ANNUITY', 'AMT_GOODS_PRICE', 'REGION_POPULATION_RELATIVE', 'DAYS_BIRTH', 'DAYS_EMPLOYED', 'DAYS_REGISTRATION', 'DAYS_ID_PUBLISH', 'OWN_CAR_AGE', 'CNT_FAM_MEMBERS', 'REGION_RATING_CLIENT', 'REGION_RATING_CLIENT_W_CITY', 'HOUR_APPR_PROCESS_START', 'REG_REGION_NOT_LIVE_REGION', 'REG_REGION_NOT_WORK_REGION', 'LIVE_REGION_NOT_WORK_REGION', 'REG_CITY_NOT_LIVE_CITY', 'REG_CITY_NOT_WORK_CITY', 'LIVE_CITY_NOT_WORK_CITY', 'EXT_SOURCE_2', 'EXT_SOURCE_3', 'OBS_30_CNT_SOCIAL_CIRCLE', 'DEF_30_CNT_SOCIAL_CIRCLE', 'OBS_60_CNT_SOCIAL_CIRCLE', 'DEF_60_CNT_SOCIAL_CIRCLE', 'DAYS_LAST_PHONE_CHANGE', 'AMT_REQ_CREDIT_BUREAU_HOUR', 'AMT_REQ_CREDIT_BUREAU_DAY', 'AMT_REQ_CREDIT_BUREAU_WEEK', 'AMT_REQ_CREDIT_BUREAU_MON', 'AM

In [26]:
##### GROUP CATEGORICAL #####
categorical_values = {}
for x in categorical_name:
    categorical_values[x] = list(data[x].unique())
categorical_values

{'NAME_TYPE_SUITE': ['Unaccompanied',
  'Family',
  'Spouse, partner',
  'Children',
  'Other_A',
  nan,
  'Other_B',
  'Group of people'],
 'NAME_INCOME_TYPE': ['Working',
  'State servant',
  'Commercial associate',
  'Pensioner',
  'Unemployed',
  'Student',
  'Businessman',
  'Maternity leave'],
 'NAME_EDUCATION_TYPE': ['Secondary / secondary special',
  'Higher education',
  'Incomplete higher',
  'Lower secondary',
  'Academic degree'],
 'NAME_FAMILY_STATUS': ['Single / not married',
  'Married',
  'Civil marriage',
  'Widow',
  'Separated',
  'Unknown'],
 'NAME_HOUSING_TYPE': ['House / apartment',
  'Rented apartment',
  'With parents',
  'Municipal apartment',
  'Office apartment',
  'Co-op apartment'],
 'OCCUPATION_TYPE': ['Laborers',
  'Core staff',
  'Accountants',
  'Managers',
  nan,
  'Drivers',
  'Sales staff',
  'Cleaning staff',
  'Cooking staff',
  'Private service staff',
  'Medicine staff',
  'Security staff',
  'High skill tech staff',
  'Waiters/barmen staff',
  '

In [27]:
for val in categorical_values['ORGANIZATION_TYPE']:
    if 'Industry' in val:
        data_rm_flag['ORGANIZATION_TYPE'].replace({val:"Industry"}, inplace=True)
    elif 'Trade' in val:
        data_rm_flag['ORGANIZATION_TYPE'].replace({val:"Trade"}, inplace=True)
    elif 'Transport' in val:
        data_rm_flag['ORGANIZATION_TYPE'].replace({val:"Transport"}, inplace=True)
    elif 'Business Entity' in val:
        data_rm_flag['ORGANIZATION_TYPE'].replace({val:"Business"}, inplace=True)

In [28]:
categorical_values = {}
for x in categorical_name:
    categorical_values[x] = list(data_rm_flag[x].unique())
categorical_values

{'NAME_TYPE_SUITE': ['Unaccompanied',
  'Family',
  'Spouse, partner',
  'Children',
  'Other_A',
  nan,
  'Other_B',
  'Group of people'],
 'NAME_INCOME_TYPE': ['Working',
  'State servant',
  'Commercial associate',
  'Pensioner',
  'Unemployed',
  'Student',
  'Businessman',
  'Maternity leave'],
 'NAME_EDUCATION_TYPE': ['Secondary / secondary special',
  'Higher education',
  'Incomplete higher',
  'Lower secondary',
  'Academic degree'],
 'NAME_FAMILY_STATUS': ['Single / not married',
  'Married',
  'Civil marriage',
  'Widow',
  'Separated',
  'Unknown'],
 'NAME_HOUSING_TYPE': ['House / apartment',
  'Rented apartment',
  'With parents',
  'Municipal apartment',
  'Office apartment',
  'Co-op apartment'],
 'OCCUPATION_TYPE': ['Laborers',
  'Core staff',
  'Accountants',
  'Managers',
  nan,
  'Drivers',
  'Sales staff',
  'Cleaning staff',
  'Cooking staff',
  'Private service staff',
  'Medicine staff',
  'Security staff',
  'High skill tech staff',
  'Waiters/barmen staff',
  '

In [29]:
data_rm_flag.to_csv('filtered_app_train_remove_flag_group_category.csv', index=False)

In [ ]:
# Correlation with TARGET
correlations = data_rm_flag[[x for x in continuous_name if x not in ['SK_ID_CURR']]].corr()['TARGET'].sort_values()
correlations_final = correlations[:-1]
correlations_final = abs(correlations_final)
correlations_final = correlations_final.sort_values()
correlations_final

In [ ]:
cor_pd = {'Features': list(correlations_final.keys()), 'Values': list(correlations_final)}
cor_pd = pd.DataFrame(cor_pd, columns = ['Features', 'Values'])
chart = sns.barplot(x = 'Values', y = 'Features', data = cor_pd,
                    label = 'Abs of Correlation coefficients', color = 'b', edgecolor = 'w')
chart.set_yticklabels(chart.get_yticklabels(), fontsize=3)

In [30]:
##### ENCODE #####
encoded_train = pd.get_dummies(data_rm_flag)
encoded_train.to_csv('encoded_train_rmflg_group.csv', index=False)

In [31]:
##### DROP NA #####
encoded_train_dropna = encoded_train.dropna(axis=0, how='any')
print(f"Before drop: {len(encoded_train)} rows\nAfter drop: {len(encoded_train_dropna)} rows")

encoded_train_dropna.to_csv('encoded_train_rmflg_group_dropna.csv', index=False)


Before drop: 307511 rows
After drop: 201084 rows


In [32]:
encoded_train.shape

(307511, 131)

In [33]:
encoded_train_dropna.shape

(201084, 131)